In [1]:
import torch
import torch.nn as nn

In [17]:
input_x = torch.rand(1,32,32,3)
input_x.shape

torch.Size([1, 32, 32, 3])

In [57]:
class ResBlock(nn.Module):
    def __init__(self):
        super(ResBlock, self).__init__()
        k3n64s1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=3//2)
        bn2d = nn.BatchNorm2d(num_features=64)
        
        self.block = nn.Sequential(
            k3n64s1,
            bn2d,
            nn.PReLU(),
            k3n64s1,
            bn2d
        )
    
    def forward(self, x):
        identity = x
        out = self.block(x)
        out += identity
        return out

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        B = 16
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=9//2),
            nn.PReLU()
        )
        
        block = [ResBlock() for _ in range(16)]
        self.block = nn.Sequential(*block)
        
        self.conv3 = = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=9//2),
            nn.BatchNorm2d(64)
        )
        
    

        
    def forward(self,x):
        x =x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x_ = x ##Residual
        x = self.block(x)
        
        return x

In [58]:
G = Generator()

In [59]:
xx = G(input_x)


xx.size()

torch.Size([1, 64, 32, 32])